In [1]:
import pandas as pd
import psycopg2
import dotenv
import os

dotenv.load_dotenv()

PG_PASSWORD = os.getenv("PG_PASSWORD")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DATABASE = os.getenv("PG_DB")

# Database connection parameters
import psycopg2
import os

conn = psycopg2.connect(
    host=PG_HOST,
    database=PG_DATABASE,
    user=PG_USER,
    password=PG_PASSWORD,
    port=PG_PORT,
    sslmode="require",
)

For all games ever

In [ ]:
import pandas as pd

# Get the distinct game_ids from the id_df
id_query = """
SELECT DISTINCT game_id 
FROM spadl_actions a
JOIN matches m ON game_id = match_id
"""
id_df = pd.read_sql_query(id_query, conn)

# Prepare the base complex query, using a placeholder for game_id
complex_query = """
WITH action_changes AS (
    SELECT
        a.*,
        LAG(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS prev_team_id,
        LEAD(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS next_team_id
    FROM
        spadl_actions a
    WHERE
        a.game_id = '{game_id}'
),
possession_markers AS (
    SELECT
        *,
        CASE WHEN prev_team_id IS NULL OR team_id != prev_team_id THEN 1 ELSE 0 END AS is_new_possession
    FROM
        action_changes
),
possession_sequences AS (
    SELECT
        *,
        SUM(is_new_possession) OVER (ORDER BY period_id, seconds, id) AS possession_group
    FROM
        possession_markers
),
possession_stats AS (
    SELECT
        possession_group,
        team_id,
        COUNT(*) AS action_count,
        MAX(id) AS last_action_id
    FROM
        possession_sequences
    GROUP BY
        possession_group, team_id
)
SELECT 
    ps.id, 
    ps.seconds, 
    ps.player_id, 
    ps.team_id, 
    t.team_name, 
    p.player_name, 
    ps.action_type, 
    ps.result, 
    ps.possession_group
FROM 
    possession_sequences ps
JOIN 
    teams t ON ps.team_id = t.team_id
JOIN 
    players p ON ps.player_id = p.player_id
"""

# Initialize an empty list to store the dataframes
all_possession_data = []

# Iterate over each game_id in id_df and execute the complex query for each game
for game_id in id_df['game_id']:
    # Substitute the game_id in the complex query
    query_with_game_id = complex_query.format(game_id=game_id)
    
    # Execute the query and get the results
    possession_df = pd.read_sql_query(query_with_game_id, conn)
    
    # Append the results to the list
    all_possession_data.append(possession_df)
    
    # Optional: Print progress
    print(f"Loaded data for game_id {game_id}, {len(possession_df)} rows")

# Concatenate all the dataframes into one
final_possession_df = pd.concat(all_possession_data, ignore_index=True)

# Save the final DataFrame to a CSV file
csv_file_path = 'possession_data.csv'
final_possession_df.to_csv(csv_file_path, index=False)

# Print confirmation
print(f"Data saved to {csv_file_path}")


For gantt matches with sequences

In [2]:
import pandas as pd

# Get the distinct game_ids from the id_df
id_query = """
SELECT DISTINCT game_id 
FROM spadl_actions a
JOIN matches m ON game_id = match_id
"""
id_df = pd.read_sql_query(id_query, conn)

# Prepare the base complex query, using a placeholder for game_id
complex_query = """
WITH action_changes AS (
    SELECT
        a.*,
        LAG(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS prev_team_id,
        LEAD(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS next_team_id
    FROM
        spadl_actions a
    WHERE
        a.game_id = '{game_id}'
),
possession_markers AS (
    SELECT
        *,
        CASE WHEN prev_team_id IS NULL OR team_id != prev_team_id THEN 1 ELSE 0 END AS is_new_possession
    FROM
        action_changes
),
possession_sequences AS (
    SELECT
        *,
        SUM(is_new_possession) OVER (ORDER BY period_id, seconds, id) AS possession_group
    FROM
        possession_markers
),
possession_stats AS (
    SELECT
        possession_group,
        team_id,
        COUNT(*) AS action_count,
        MAX(id) AS last_action_id
    FROM
        possession_sequences
    GROUP BY
        possession_group, team_id
)
SELECT 
    ps.id, 
    ps.seconds, 
    ps.player_id, 
    ps.team_id, 
    t.team_name, 
    p.player_name, 
    ps.action_type, 
    ps.result, 
    ps.possession_group
FROM 
    possession_sequences ps
JOIN 
    teams t ON ps.team_id = t.team_id
JOIN 
    players p ON ps.player_id = p.player_id
"""

# Initialize an empty list to store the dataframes
all_possession_data = []

# Iterate over each game_id in id_df and execute the complex query for each game
for game_id in id_df['game_id']:
    # Substitute the game_id in the complex query
    query_with_game_id = complex_query.format(game_id=game_id)
    
    # Execute the query and get the results
    possession_df = pd.read_sql_query(query_with_game_id, conn)
    
    # Filter the results to include only the rows where team_id is '8y3iucyxguipljcmf87a11bk9'
    possession_df = possession_df[possession_df['team_id'] == '8y3iucyxguipljcmf87a11bk9']
    
    # Append the filtered results to the list
    all_possession_data.append(possession_df)
    
    # Optional: Print progress
    print(f"Loaded data for game_id {game_id}, {len(possession_df)} rows after filtering")

# Concatenate all the dataframes into one
final_possession_df = pd.concat(all_possession_data, ignore_index=True)

# Save the final DataFrame to a CSV file
csv_file_path = 'possession_data_filtered.csv'
final_possession_df.to_csv(csv_file_path, index=False)

# Print confirmation
print(f"Data saved to {csv_file_path}")


C:\Users\Gytis\AppData\Local\Temp\ipykernel_13228\899566789.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  id_df = pd.read_sql_query(id_query, conn)
C:\Users\Gytis\AppData\Local\Temp\ipykernel_13228\899566789.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  possession_df = pd.read_sql_query(query_with_game_id, conn)


Loaded data for game_id 5oc8drrbruovbuiriyhdyiyok, 0 rows after filtering
Loaded data for game_id 5ow2wa823rjft38oh48b4ror8, 0 rows after filtering
Loaded data for game_id 5pcyhm34h5c948yji4oryevpw, 0 rows after filtering
Loaded data for game_id 5ptnar4qtaltvcfsjdw9vzhg4, 0 rows after filtering
Loaded data for game_id 5qa6smqwjobwdm3j0okr0r09g, 0 rows after filtering
Loaded data for game_id 5qqz2do47zjms7rfrwzcths7o, 1097 rows after filtering
Loaded data for game_id 5r7lyj3frtml3aqy458jrxj4k, 0 rows after filtering
Loaded data for game_id 5ro450tmjg6o1r4cphzzyg74k, 0 rows after filtering
Loaded data for game_id 5sue6595uizdrssh0wv22egpg, 0 rows after filtering
Loaded data for game_id 5t7w35uh3t29uk7t0egu7q784, 0 rows after filtering
Loaded data for game_id 5tle8ulixbd5vc0y4p7myvrbo, 0 rows after filtering
Loaded data for game_id 5u35bb21cuumrm2th8czyecr8, 0 rows after filtering
Loaded data for game_id 5ugfsw7je1y0lay7xdqe3pces, 0 rows after filtering
Loaded data for game_id 5uts2s7fl98

File maker for "period_id","seconds","start_x","start_y","end_x","end_y","action_type","result","possession_group"

In [ ]:
import pandas as pd

# Get the distinct game_ids from the id_df
id_query = """
SELECT DISTINCT game_id 
FROM spadl_actions a
JOIN matches m ON game_id = match_id
"""
id_df = pd.read_sql_query(id_query, conn)

# Prepare the base complex query, using a placeholder for game_id
complex_query = """
WITH action_changes AS (
    SELECT
        a.*,
        LAG(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS prev_team_id,
        LEAD(a.team_id) OVER (ORDER BY a.period_id, a.seconds, a.id) AS next_team_id
    FROM
        spadl_actions a
    WHERE
        a.game_id = '{game_id}'
),
possession_markers AS (
    SELECT
        *,
        CASE WHEN prev_team_id IS NULL OR team_id != prev_team_id THEN 1 ELSE 0 END AS is_new_possession
    FROM
        action_changes
),
possession_sequences AS (
    SELECT
        *,
        SUM(is_new_possession) OVER (ORDER BY period_id, seconds, id) AS possession_group
    FROM
        possession_markers
),
possession_stats AS (
    SELECT
        possession_group,
        team_id,
        COUNT(*) AS action_count,
        MAX(id) AS last_action_id
    FROM
        possession_sequences
    GROUP BY
        possession_group, team_id
)
SELECT 
    ps.period_id,
    ps.seconds, 
    ps.start_x,
    ps.start_y, 
    ps.end_x, 
    ps.end_y, 
    ps.action_type, 
    ps.result, 
    ps.possession_group
FROM 
    possession_sequences ps
JOIN 
    teams t ON ps.team_id = t.team_id
JOIN 
    players p ON ps.player_id = p.player_id
"""

# Initialize an empty list to store the dataframes
all_possession_data = []

# Iterate over each game_id in id_df and execute the complex query for each game
for game_id in id_df['game_id']:
    # Substitute the game_id in the complex query
    query_with_game_id = complex_query.format(game_id=game_id)
    
    # Execute the query and get the results
    possession_df = pd.read_sql_query(query_with_game_id, conn)
    
    # Filter the results to include only the rows where team_id is '8y3iucyxguipljcmf87a11bk9'
    possession_df = possession_df[possession_df['team_id'] == '8y3iucyxguipljcmf87a11bk9']
    
    # Append the filtered results to the list
    all_possession_data.append(possession_df)
    
    # Optional: Print progress
    print(f"Loaded data for game_id {game_id}, {len(possession_df)} rows after filtering")

# Concatenate all the dataframes into one
final_possession_df = pd.concat(all_possession_data, ignore_index=True)

# Save the final DataFrame to a CSV file
csv_file_path = 'possession_data_filtered.csv'
final_possession_df.to_csv(csv_file_path, index=False)

# Print confirmation
print(f"Data saved to {csv_file_path}")
